In [1]:
from databroker.v2 import temp
from bluesky import RunEngine, Msg
from bluesky.plans import count
import bluesky.plan_stubs as bps

from secop_ophyd.SECoPDevices import SECoP_Node_Device, SECoPReadableDevice, SECoPMoveableDevice, SECoP_CMD_Device
import asyncio
import random



# Create a run engine and a temporary file backed database. Send all the documents from the RE into that database
RE = RunEngine({})
db = temp()
RE.subscribe(db.v1.insert)

from bluesky.plans import scan
from ophyd.sim import det, motor

#Example of adding metadata to RE environment

investigation_id = "kmnk2n34"

RE.md["investigation_id"] = investigation_id

robo_dev = await SECoP_Node_Device.create('localhost','10770',RE.loop)

# now do something
RE(scan([det],motor,1,2,10),reason="for a demo")

storage:SECoPReadableDevice= robo_dev.storage
sample:SECoPMoveableDevice= robo_dev.sample
loadshort:SECoP_CMD_Device = storage.load_short_dev
measure:SECoP_CMD_Device = sample.measure_dev


sample_changer.HZB ready


In [2]:
await robo_dev.read_configuration()



reading = await storage.value.read()


reading = await storage.value.read()
print(reading)


loadshort:SECoP_CMD_Device = storage.load_short_dev

for samplepos in range(1,13):
    await loadshort.samplepos_arg.set(samplepos)
    await loadshort.substance_arg.set(random.randint(0,6))

    await loadshort.load_short_x.execute()

    await storage.wait_for_IDLE()
    print(samplepos)
        
        

{'sample_changer-HZB-storage-value': {'value': [], 'timestamp': 1693600582.9785001}}
1
2
3
4
5
6
7
8
9
10
11
12


In [ ]:
await robo_dev.storage.read()
#await robo_dev.storage.read_configuration()

RE(count([storage],delay=2,num=5))


In [2]:


def measure(detectors,sample,sample_num):
    reading = yield from bps.read(sample)
    print(reading)
    
    curr_sample = reading[sample.value.name]['value']
    
    if curr_sample != 0 and curr_sample  != sample_num :
        yield from bps.mv(sample,0)
    
    
    if curr_sample is not sample_num:
        yield from bps.mv(sample,i)
    
    # Get the status object that tells us when it's done
    uid =  yield from bps.open_run()
    complete_status = yield from bps.complete(measure, wait=False) #This message doesn't exist yet
 
    # While the device is still executing, read from the detectors in the detectors list
    while not complete_status.done:

        yield Msg('checkpoint') # allows us to pause the run 
        yield from bps.one_shot(detectors) #triggers and reads everything in the detectors list
        yield Msg('sleep', None, 1)
        
    
    yield bps.close_run()
    
    yield from  RE(bps.mv(sample,0))

    return uid




for i in range(1,13):
    #grab sample i and hold in Measurement Pos
    RE(measure(det,sample,i))
    

    


Run aborted
Traceback (most recent call last):
  File "/home/peter/git-repos/secop-ophyd/.venv/lib64/python3.11/site-packages/bluesky/run_engine.py", line 1503, in _run
    msg = self._plan_stack[-1].throw(stashed_exception or resp)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_70119/1754883000.py", line 16, in measure
    complete_status = yield from bps.complete(measure, wait=False) #This message doesn't exist yet
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/peter/git-repos/secop-ophyd/.venv/lib64/python3.11/site-packages/bluesky/plan_stubs.py", line 657, in complete
    ret = yield Msg('complete', obj, group=group, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/peter/git-repos/secop-ophyd/.venv/lib64/python3.11/site-packages/bluesky/run_engine.py", line 1588, in _run
    new_response = await coro(msg)
                   ^^^^^^^^^^^^^^^
  File "/home/peter/git-repos/secop

{'sample_changer-HZB-sample-value': {'value': 1, 'timestamp': 1693602142.201729}}


AssertionError: <function measure at 0x7fd76a4b2d40> does not implement all Flyable methods